In [1]:
USE TargetMartII;
GO

/*********************************************************/
/******************    Schema DDL       ******************/
/*********************************************************/

IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'dim' ) 
BEGIN
	EXEC sp_executesql N'CREATE SCHEMA dim AUTHORIZATION dbo;'
END
;

GO

IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'f' ) 
BEGIN
	EXEC sp_executesql N'CREATE SCHEMA f AUTHORIZATION dbo;'
END
;

GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.034

In [12]:
USE TargetMartII;
GO

/*********************************************************/
/******************  Customer DIM DDL   ******************/
/*********************************************************/

IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Customers')
BEGIN
	CREATE TABLE dim.Customers(
	pkCustomer int IDENTITY(1000,1) NOT NULL,
	CustomerID nvarchar(5) NOT NULL,
	Customer nvarchar(40) NOT NULL,
	City nvarchar(15) NULL,
	Country nvarchar(15) NULL,
	LoadDate DATE NOT NULL,
	SourceCountry nvarchar(25) NULL
	)
	;

	ALTER TABLE dim.Customers
	ADD CONSTRAINT PK_Customers_LUP PRIMARY KEY(pkCustomer);

	ALTER TABLE dim.Customers
    ADD CONSTRAINT UC_Customers_ID UNIQUE (CustomerID);

END


GO


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.001

In [2]:
/*********************************************************/
/****************** Calendar DIM Script ******************/
/*********************************************************/

IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Calendar')
BEGIN
-- Create the Calendar table
CREATE TABLE dim.Calendar
(
    pkCalendar INT NOT NULL,
    DateValue DATE NOT NULL,
    Year INT NOT NULL,
    Quarter INT NOT NULL,
	Qtr VARCHAR(3) NOT NULL,
    Month INT NOT NULL,
    MonthName VARCHAR(10) NOT NULL,
	MonthShort VARCHAR(3) NOT NULL,
    Week INT NOT NULL,
    Day INT NOT NULL,
	DayName VARCHAR(10) NOT NULL,
	DayShort VARCHAR(3) NOT NULL,
    IsWeekday BIT NOT NULL,
	Weekday VARCHAR(3) NOT NULL
);

	ALTER TABLE dim.Calendar
	ADD CONSTRAINT PK_Calendar_Julian PRIMARY KEY(pkCalendar);

	ALTER TABLE dim.Calendar
    ADD CONSTRAINT UC_Calendar UNIQUE (DateValue);
END

GO


Commands completed successfully.

Total execution time: 00:00:00.028

In [3]:
/*********************************************************/
/******************  Shipper DIM DDL    ******************/
/*********************************************************/
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Shipper')
BEGIN
-- Create the Calendar table
CREATE TABLE dim.Shipper
(
    pkShipId int not null,
	Shipper nvarchar(40) not null,
	Phone nvarchar(24) null
);

	ALTER TABLE dim.Shipper
	ADD CONSTRAINT PK_Ship PRIMARY KEY(pkShipId);
END

GO

/*********************************************************/
/******************  Products DIM DDL   ******************/
/*********************************************************/

IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Products')
BEGIN
CREATE TABLE dim.Products(
	pkProdId int NOT NULL,
	ProductId int NOT NULL,
	Product nvarchar(50) NOT NULL,
	ProductCategory nvarchar(25) NOT NULL,
	CategoryDesc ntext NULL,
	UnitPrice money NULL,
	UnitsInStock int NULL,
	UnitsOnOrder int NULL,
	ReorderLevel int NULL,
	Discontinued bit NOT NULL,
	ReorderFlag nvarchar(3) NOT NULL
);

	ALTER TABLE dim.Products
	ADD CONSTRAINT PK_Prods PRIMARY KEY(pkProdId);

	ALTER TABLE dim.Products
    ADD CONSTRAINT UC_Prods UNIQUE (ProductId);
END
;
GO

/*********************************************************/
/******************  Employees DIM DDL   ******************/
/*********************************************************/
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Employees')
BEGIN
CREATE TABLE dim.Employees(
	EmployeeID int NOT NULL,
	Employee nvarchar(50) NOT NULL,
	Title nvarchar(30) NULL,
	BirthDate datetime NULL,
	HireDate datetime NULL,
	City nvarchar(15) NULL,
	Country nvarchar(15) NULL,
	ReportsTo int NULL
);
	ALTER TABLE dim.Employees
	ADD CONSTRAINT PK_Emp PRIMARY KEY(EmployeeID);

END

GO

/*********************************************************/
/*********************************************************/
/*********************************************************/
/******************  Fact Table Builds  ******************/
/*********************************************************/
/*********************************************************/
/*********************************************************/



/*********************************************************/
/******************  OrderPerf f.Table  ******************/
/*********************************************************/

-- OrderID, OrderDate, Customer, Freight(f)
IF EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'f' AND TABLE_NAME = 'OrderPerf')
BEGIN 
	DROP TABLE f.OrderPerf;
END

GO

CREATE TABLE f.OrderPerf(
	OrderID int NOT NULL,
	fkCalendar int NOT NULL,
	fkCustomer int NOT NULL,
	fkShipper int NULL,
	fkEmployee int NULL,
	Freight money NOT NULL,
	DaysToShip int NULL,
	DaysTilRequired int NULL,
	OrderCount int NOT NULL
);

-- Could use PRIMARY or UNIQUE for the ORDERID - just used to not load dupes
ALTER TABLE f.OrderPerf
ADD CONSTRAINT PK_ORD PRIMARY KEY(OrderID)
;
GO


/*********************************************************/
/******************  ProductsPerf f.Table  ******************/
/*********************************************************/

IF EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'f' AND TABLE_NAME = 'ProductPerf')
BEGIN 
	DROP TABLE f.ProductPerf;
END

CREATE TABLE f.ProductPerf(
	OrderID int NOT NULL,
	OrderDate int NOT NULL,
	ProductID int NOT NULL,
	CustomerID int NOT NULL,
	UnitPrice money NOT NULL,
	Quantity smallint NOT NULL,
	Discount float NOT NULL,
	DiscFlag int NOT NULL,
	LineTotal money NULL
);


ALTER TABLE f.ProductPerf
ADD CONSTRAINT PK_ProdPerf PRIMARY KEY(OrderID, ProductID)
;
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.128

**Data Loading**

<span style="color: var(--vscode-foreground);">Data from Northwind2023</span>

In [4]:
USE TargetMartII;
GO

/*
  Load the customer dimension data ...
*/

TRUNCATE TABLE TargetMartII.dim.Customers;
GO

INSERT INTO dim.Customers(CustomerID, Customer, City, Country, LoadDate, SourceCountry)
	SELECT nwc.CustomerID
		  ,nwc.CompanyName 
		  ,nwc.City
		  ,nwc.Country
		  ,CAST(getdate() as DATE)
		  ,concat('Country = ', nwc.Country) 
	FROM Northwind_2023.dbo.Customers nwc
	WHERE nwc.CustomerID not in (SELECT CustomerID FROM dim.Customers)

;
GO

/*
  Load the calendar dimension data ...
*/

IF (SELECT count(*) FROM dim.Calendar) = 0
BEGIN
-- Declare variables
DECLARE @StartDate DATE = '2020-01-01'
DECLARE @EndDate DATE = DATEADD(year, 0, GETDATE())
DECLARE @Date DATE = @StartDate
DECLARE @DayID INT = (datepart(year, @StartDate)-1900)*1000 + datepart(dy, @StartDate)
;

-- Populate the Calendar table
WHILE @Date <= @EndDate
	BEGIN
		INSERT INTO dim.Calendar (pkCalendar, DateValue, Year, Quarter, Qtr, Month,  MonthName, MonthShort, Week, Day, DayName, DayShort, IsWeekday, Weekday)
		VALUES (
			@DayID,
			@Date,
			YEAR(@Date),
			DATEPART(QUARTER, @Date),
			CASE WHEN DATEPART(QUARTER, @Date) IN (1) THEN '1st'
				 WHEN DATEPART(QUARTER, @Date) IN (2) THEN '2nd'
				 WHEN DATEPART(QUARTER, @Date) IN (3) THEN '3rd'
				 WHEN DATEPART(QUARTER, @Date) IN (4) THEN '4th'
				 ELSE '5th'
				 END,
			MONTH(@Date),
			DATENAME(MONTH, @Date),
			LEFT(DATENAME(MONTH, @Date),3),
			DATEPART(WEEK, @Date),
			DAY(@Date),
			DATENAME(WEEKDAY, @Date),
			LEFT(DATENAME(WEEKDAY, @Date),3),
			CASE WHEN DATEPART(WEEKDAY, @Date) IN (1, 7) THEN 0 ELSE 1 END, -- Set IsWeekday to 0 for Saturday (1) and Sunday (7), and 1 for weekdays
			CASE WHEN DATEPART(WEEKDAY, @Date) IN (1, 7) THEN 'No' ELSE 'Yes' END
	 )

		-- Increment the date and day ID
		SET @Date = DATEADD(DAY, 1, @Date)
		SET @DayID = @DayID + 1
	END
END
GO


/*
  Load the shipper dimension data ...
*/

TRUNCATE TABLE TargetMartII.dim.Shipper;
GO

INSERT INTO dim.Shipper(pkShipId, Shipper, Phone)
SELECT s_sh.ShipperID
      ,s_sh.CompanyName
      ,s_sh.Phone
FROM Northwind_2023.dbo.Shippers s_sh
WHERE s_sh.ShipperID not in (SELECT pkShipId FROM dim.Shipper)
;
GO

/*
  Load the products dimension data ...
*/

TRUNCATE TABLE TargetMartII.dim.Products;
GO

INSERT INTO dim.Products(pkProdId, ProductId, Product, ProductCategory, CategoryDesc, UnitPrice, UnitsInStock, UnitsOnOrder, ReorderLevel, Discontinued, ReorderFlag)
SELECT prod.ProductID + 10000 as 'pkProdId'
      , prod.ProductID as 'ProductId'
      , prod.ProductName as 'Product'
	  , cat.CategoryName as 'ProductCategory'
	  , cat.[Description] as 'CategoryDesc'
	  , prod.UnitPrice
	  , prod.UnitsInStock
	  , prod.UnitsOnOrder
	  , prod.ReorderLevel
	  , prod.Discontinued
	  , CASE 
			WHEN prod.ReorderLevel >= (prod.UnitsInStock + prod.UnitsOnOrder)
			     AND prod.Discontinued = 0
			 THEN 'Yes'
	      ELSE 'No'
	    END as 'ReorderFlag'
FROM Northwind_2023.dbo.Products prod
	INNER JOIN Northwind_2023.dbo.Categories cat
	ON prod.CategoryID = cat.CategoryID
WHERE prod.ProductID not in (SELECT ProductId FROM dim.Products)
;
GO

/*
  Load the employees dimension data ...
*/

TRUNCATE TABLE TargetMartII.dim.Employees;
GO

INSERT INTO dim.Employees(EmployeeID, Employee, Title, BirthDate, HireDate, City, Country, ReportsTo)
SELECT sEmp.EmployeeID
	  ,concat( sEmp.TitleOfCourtesy, ' ', sEmp.FirstName, ', ', sEmp.LastName)
      ,sEmp.Title
      ,sEmp.BirthDate
      ,sEmp.HireDate
      ,sEmp.City
      ,sEmp.Country
      ,sEmp.ReportsTo
FROM Northwind_2023.dbo.Employees sEmp
WHERE sEmp.EmployeeID not in (SELECT EmployeeID FROM dim.Employees)
;
GO


/*********************************************************/
/******************  Fact TableLoaders  ******************/
/*********************************************************/


/******************  Orders Perf Fact   ******************/

TRUNCATE TABLE TargetMartII.f.OrderPerf;
GO

INSERT INTO TargetMartII.f.OrderPerf(OrderID, fkCalendar, fkCustomer, fkShipper, fkEmployee, Freight, DaysToShip, DaysTilRequired, OrderCount)
SELECT sO.OrderID
	  ,tC.pkCalendar as fkCalendar
	  ,tCus.pkCustomer as fkCustomer
	  ,sO.ShipVia as fkShipper
	  ,sO.EmployeeID as fkEmployee
	  ,sO.Freight
	  ,DATEDIFF ( day , sO.OrderDate , sO.ShippedDate ) as 'DaysToShip'
	  ,DATEDIFF ( day , sO.OrderDate , sO.RequireDate ) as 'DaysTilRequired'
	  , 1 as 'OrderCount'
FROM Northwind_2023.dbo.Orders sO
	INNER JOIN TargetMartII.dim.Calendar tC
	ON cast(so.OrderDate as DATE) = tc.DateValue
	INNER JOIN TargetMartII.dim.Customers tCus
	ON sO.CustomerID = tCus.CustomerID
; 
GO

/******************  Products Perf Fact  ******************/

TRUNCATE TABLE TargetMartII.f.ProductPerf;
GO

INSERT INTO TargetMartII.f.ProductPerf(OrderID, OrderDate, ProductID, CustomerID, UnitPrice, Quantity, Discount, DiscFlag, LineTotal)
SELECT sDet.OrderID
      ,tCal.pkCalendar as 'OrderDate'
      ,sDet.ProductID + 10000 as 'ProductID'
--	  ,sOrd.CustomerID
	  ,tCust.pkCustomer
      ,sDet.UnitPrice
      ,sDet.Quantity
      ,sDet.Discount
	  ,CASE 
		 WHEN sDet.Discount = 0 THEN 0
		 ELSE 1
	   END as 'DiscFlag'
	  ,CAST(((1 - sDet.Discount) * sDet.UnitPrice) * sDet.Quantity as MONEY) as 'LineTotal'
FROM Northwind_2023.dbo.OrderDetails sDet
	INNER JOIN Northwind_2023.dbo.Orders sOrd
	ON sDet.OrderID = sOrd.OrderID
	INNER JOIN dim.Calendar tCal
	ON CAST(sOrd.OrderDate as DATE) = tCal.DateValue
	INNER JOIN dim.Customers tCust
	ON sOrd.CustomerID = tCust.CustomerID
;
GO

Commands completed successfully.

Commands completed successfully.

(91 rows affected)

Commands completed successfully.

Commands completed successfully.

...

Commands completed successfully.

(830 rows affected)

Commands completed successfully.

(2155 rows affected)

Total execution time: 00:00:00.431

![](c:\Users\biauser\Desktop\Repo.PNG)